In [7]:
from __future__ import print_function
import pandas as pd
import matplotlib.pyplot as plt

from lime.lime_tabular import RecurrentTabularExplainer
from tqdm import tqdm
import keras
from sp_modif.model_function import *
from sp_modif.methods import *
from sp_modif.data_prep import *
from sp_modif.evaluator import *
from sp_modif.SHAP import *
from sp_modif.L2X import *
import warnings

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import random
# import keras
import math

from sklearn.metrics import mean_squared_error, r2_score 
from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
from keras import backend as K
from sklearn.preprocessing import MinMaxScaler , StandardScaler
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Activation, GRU
from scipy import optimize
from methods import *
import warnings
from tensorflow.keras import optimizers
warnings.filterwarnings("ignore")
%matplotlib inline

warnings.filterwarnings('ignore')
print("okay")

okay


### Functions

In [8]:
# Function to generate explanations
# Function explanation for lime

def get_lime_explanation(data, e, L2X=False) :
    # e  = fn = model.predict
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(data.shape[1]*data.shape[2])])

    # Get explanations
    for row in range(data.shape[0]) : 
        explanation = lime_explainer.explain_instance(data[row],
                                                      e,
                                                      num_features=data.shape[1]*data.shape[2]) 
        # fn = model.predict, initialize lime_explainer = Reccurent()
        lime_values = explanation.local_exp[1]
        # Add explanation in df_explanation
        lime_dict = {}
        for tup in lime_values :
            lime_dict[str(tup[0])] = tup[1]
        df_expplanation.loc[len(df_expplanation)] = lime_dict
    
    return df_expplanation

# # Function explanation for others
def get_explainations(data, e, L2X = False) :
    
    # df diemnsion
    if L2X==True :
        X_to_def_col = data[0:1]
        explanation_test = e.explain(X_to_def_col.reshape((X_to_def_col.shape[0], -1)))
        num_columns = explanation_test.flatten().shape[0]
        
    else : 
        explanation_test = e.explain(data[0:1])
        num_columns = explanation_test.flatten().shape[0]
    
    # Iniatialisation
    df_expplanation = pd.DataFrame(columns=[str(i) for i in range(num_columns)])

    # Get explanations
    for row in range(data.shape[0]) :
        if L2X==True:
            X_row = data[row:row+1]
            explanation = e.explain(X_row.reshape((X_row.shape[0], -1)))
        else :
            explanation = e.explain(data[row:row+1])
        # Add explanation in df_explanation
        explanation = explanation.flatten()
        feature_dict = {}
        for i in range(num_columns) :
            feature_dict[str(i)] = explanation[i]
        df_expplanation.loc[len(df_expplanation)] = feature_dict
    
    return df_expplanation

### FD001

In [9]:
# Load data and preprocess
train, test, y_test = prepare_data('FD001.txt')
print(train.shape, test.shape, y_test.shape)
sensor_names = ['T20','T24','T30','T50','P20','P15','P30','Nf','Nc','epr','Ps30','phi',
                'NRf','NRc','BPR','farB','htBleed','Nf_dmd','PCNfR_dmd','W31','W32']

remaining_sensors = ['T24','T30','T50','P30','Nf','Nc','Ps30','phi',
                'NRf','NRc','BPR','htBleed','W31','W32'] # selection based on main_notebook

drop_sensors = [element for element in sensor_names if element not in remaining_sensors]

(20631, 27) (13096, 26) (100, 1)


In [13]:
# 1layers
def model_gru_1layer_optimal(input_shape, weights_file):
    
    nodes_per_layer = 256
    dropout = 0.3
    activation = 'tanh'
    
    cb = keras.callbacks.EarlyStopping(monitor='loss', patience=4)
    model = Sequential()
    model.add(GRU(units = nodes_per_layer, activation=activation, 
                  input_shape=input_shape))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))
    model.save_weights(weights_file)

    return model

In [15]:
from tqdm import tqdm
sequence_lengths = [30, 35, 38, 39, 40]
list_alpha = [0.2, 0.3, 0.5]
list_upper = [120,125,130]
metrics_names = ['identity', 'separability', 'stability', 'coherence', 'completness', 'congruence',
                 'selectivity', 'accumen',	'Verm_stability',	'fidelity',	'sparsity', 'instability']
                     
df_metrics = pd.DataFrame()
# all_list = [(seq, alpha, upper) for seq in sequence_lengths for alpha in list_alpha for rec_rul in list_upper]
all_list = []
for sq in sequence_lengths:
    for alpha in list_alpha:
        for upper in list_upper:
            triplet = (sq, alpha, upper)
            all_list.append(triplet)
list_d = []
list_df_metric = []

for elm in tqdm(all_list):
    # Modeling
    print(elm)
    print('...')
    sequence_length = elm[0]
    alpha = elm[1]
    upper = elm[2]
    learning_rate_ = 0.001
    activation = 'tanh'
    epochs = 20
    batch_size = 128
    train = rul_piecewise_fct(train, upper)

    # Model creation
    weights_file = str(alpha) + '1gru_hyper_parameter_weights.h5'
    input_shape = (sequence_length, len(remaining_sensors))
    model = model_gru_1layer_optimal(input_shape=input_shape, weights_file = weights_file)
    mse = []

    # Data prepration
    X_train_interim, X_test_interim = prep_data(train, test, drop_sensors, remaining_sensors, alpha)

    # create sequences train, test
    train_array = gen_data_wrapper(X_train_interim, sequence_length,remaining_sensors)
    label_array = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'])

    test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
               for unit_nr in X_test_interim['Unit'].unique())
    
    test_array = np.concatenate(list(test_gen)).astype(np.float32)
    print(train_array.shape, label_array.shape, test_array.shape)

    # create train-val split
    gss = GroupShuffleSplit(n_splits=3, train_size=0.80, random_state=42)
    
    mse_val = []
    R2_val = []
    RMSE = []
    score_val = []
    test_rul = rul_piecewise_fct(y_test,upper)
    
    with tf.device('/device:GPU:0'):
        for train_unit, val_unit in gss.split(X_train_interim['Unit'].unique(), groups=X_train_interim['Unit'].unique()):
            train_unit = X_train_interim['Unit'].unique()[train_unit]  # gss returns indexes and index starts at 1
            train_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, train_unit)
            train_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], train_unit)

            val_unit = X_train_interim['Unit'].unique()[val_unit]
            val_split_array = gen_data_wrapper(X_train_interim, sequence_length, remaining_sensors, val_unit)
            val_split_label = gen_label_wrapper(X_train_interim, sequence_length, ['RUL'], val_unit)

            # train and evaluate model
            model.compile(loss='mse', optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001))
#             model.load_weights(weights_file)  # reset optimizer and node weights before every training iteration

            # with tf.device('/device:GPU:0'):
            history = model.fit(train_split_array, train_split_label,
                                validation_data=(val_split_array, val_split_label),
                                epochs=epochs,
                                batch_size=batch_size,
                                verbose=0)
#             mse.append(history.history['val_loss'][-1])
            test_gen = (list(gen_test_data(X_test_interim[X_test_interim['Unit']==unit_nr], sequence_length,remaining_sensors, -99.))
                            for unit_nr in X_test_interim['Unit'].unique())
            test_array = np.concatenate(list(test_gen)).astype(np.float32)
            mse_val.append(history.history['val_loss'][-1])
            y_hat_val_split = model.predict(test_array)
            R2_val.append(r2_score(test_rul, y_hat_val_split))
            RMSE.append(np.sqrt(mean_squared_error(test_rul, y_hat_val_split)))
            score_val.append(compute_s_score(test_rul, y_hat_val_split))
    d = {'RMSE_val':np.sqrt(mse_val),'RMSE_test': RMSE,'R2_test':np.mean(R2_val), 'Score_test':np.mean(score_val),
                'alpha':alpha, 'rul_piecewise':upper, 'sequence_length':sequence_length}
    list_d.append(d)
    
    
    # Get explanation and calculate the score
    # Echantillonage
    n_individus = test_array.shape[0]

    # # Choisir aléatoirement 10 indices d'individus
    np.random.seed(0)
    indices_choisis = np.random.choice(n_individus, size=10, replace=False)

    # Sélectionner les données correspondant aux indices choisis
    test_array_sampling = test_array[indices_choisis, :, :]
    label_array_sampling = y_test.values[indices_choisis, :]

    # Afficher les dimensions des données sélectionnées
    print(test_array_sampling.shape, label_array_sampling.shape)
    
    # distance matrix XX'
    X_dist = pd.DataFrame(squareform(pdist(test_array_sampling.reshape((test_array_sampling.shape[0], -1)))))

    # LIME
    lime_explainer = RecurrentTabularExplainer(test_array, training_labels=label_array,
                                                       feature_names=remaining_sensors,
                                                       mode = 'regression',
                                                       )
    lime_values = get_lime_explanation(test_array_sampling, e = model.predict)
    lime_values.shape
    Lime_dist = pd.DataFrame(squareform(pdist(lime_values))) # Lime values explanation matrix

    #Lime's metrics
    list_metrics_lime = {}
    list_metrics_lime['identity'] = identity(X_dist, Lime_dist)
    list_metrics_lime['separability'] = separability(X_dist, Lime_dist)
    list_metrics_lime['stability'] = stability(X_dist, Lime_dist)
    list_metrics_lime['coherence'], list_metrics_lime['completness'], list_metrics_lime['congruence'] = coherence(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                   samples=test_array_sampling,
                                                    targets=label_array, e = model.predict)
    list_metrics_lime['selectivity'] = selectivity(model=model.predict, explainer = get_lime_explanation,
                                                   samples=test_array_sampling, e_x = model.predict)
    list_metrics_lime['accumen'] = acumen(get_lime_explanation, test_array_sampling, e=model.predict)
    list_metrics_lime['Verm_stability'] = stability_Velmurugan(get_lime_explanation, test_array_sampling,
                                                                   e=model.predict, top_features=200)
    list_metrics_lime['fidelity'], list_metrics_lime['sparsity'] = fidelity(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                    samples=test_array_sampling,
                                                    e = model.predict, L2X=True)
    list_metrics_lime['instability'] = instability(model=model.predict, 
                                                    explainer = get_lime_explanation,
                                                    samples=test_array_sampling,
                                                    e = model.predict, L2X=True)
    
    list_metrics_lime['alpha'] = alpha
    list_metrics_lime['upper'] = upper
    list_metrics_lime['sequence_length'] = sequence_length
    list_metrics_lime['explainer'] = 'lime'
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_lime])])
    
    # SHAP
    e = KernelSHAP(model)
    shapvalues = get_explainations(test_array_sampling, e)
    shapvalues.shape
    list_metrics_shap = {}
    shap_dist = pd.DataFrame(squareform(pdist(shapvalues))) # shap values explanation matrix
    list_metrics_shap['identity'] = identity(X_dist, shap_dist)
    list_metrics_shap['separability'] = separability(X_dist, shap_dist)
    list_metrics_shap['stability'] = stability(X_dist, shap_dist)
    list_metrics_shap['coherence'], list_metrics_shap['completness'], list_metrics_shap['congruence'] = coherence(model=model.predict, 
                                                    explainer = get_explainations,
                                                   samples=test_array_sampling,
                                                    targets=label_array, e = e)
    list_metrics_shap['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                       samples=test_array_sampling, e_x=e)
    list_metrics_shap['accumen'] = acumen(get_explainations, test_array_sampling, e=e)
    list_metrics_shap['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                   e=e, top_features=200)
    list_metrics_shap['fidelity'], list_metrics_shap['sparsity']= fidelity(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e)
    list_metrics_shap['instability']= instability(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e)
    
    list_metrics_shap['alpha'] = alpha
    list_metrics_shap['upper'] = upper
    list_metrics_shap['sequence_length'] = sequence_length
    list_metrics_shap['explainer'] = 'shap'

    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_shap])])
    
    # L2X
    e = L2X(model.predict, test_array_sampling)
    l2xvalues = get_explainations(test_array_sampling, e, L2X=True)
    l2xvalues.shape

    # L2X's metrics
    list_metrics_l2x = {}
    l2x_dist = pd.DataFrame(squareform(pdist(l2xvalues))) # l2x values explanation matrix

    list_metrics_l2x['identity'] = identity(X_dist, l2x_dist)
    list_metrics_l2x['separability'] = separability(X_dist, l2x_dist)
    list_metrics_l2x['stability'] = stability(X_dist, l2x_dist)
    list_metrics_l2x['coherence'], list_metrics_l2x['completness'], list_metrics_l2x['congruence'] = coherence(model=model.predict, explainer = get_explainations,
                                                   samples=test_array_sampling, targets=label_array_sampling, e = e, L2X=True)
    list_metrics_l2x['selectivity'] = selectivity(model=model.predict, explainer = get_explainations,
                                       samples=test_array_sampling, e_x=e, L2X=True)
    list_metrics_l2x['accumen'] = acumen(get_explainations, test_array_sampling, e=e, L2X=True)
    list_metrics_l2x['Verm_stability'] = stability_Velmurugan(get_explainations, test_array_sampling,
                                                                   e=e, top_features=200, L2X=True)
    list_metrics_l2x['fidelity'], list_metrics_l2x['sparsity']= fidelity(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e, L2X=True)
    list_metrics_l2x['instability'] = instability(model=model.predict, 
                                                    explainer = get_explainations,
                                                    samples=test_array_sampling,
                                                    e = e, L2X=True)
    
    list_metrics_l2x['alpha'] = alpha
    list_metrics_l2x['upper'] = upper
    list_metrics_l2x['sequence_length'] = sequence_length
    list_metrics_l2x['explainer'] = 'l2x'

    
    df_metrics = pd.concat([df_metrics, pd.DataFrame([list_metrics_l2x])])

  0%|          | 0/45 [00:00<?, ?it/s]

(30, 0.2, 120)
...
(17731, 30, 14) (17731, 1) (100, 30, 14)
4/4 [==============================] - 0s 6ms/step
(10, 30, 14) (10, 1)
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
1

  2%|▏         | 1/45 [2:05:38<92:08:06, 7538.32s/it]

(30, 0.2, 125)
...
(17731, 30, 14) (17731, 1) (100, 30, 14)
4/4 [==============================] - 0s 14ms/step
(10, 30, 14) (10, 1)
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 2s 10ms/step
10
157/157 [==============================] - 2s 10ms/step
10
157/157 [==============================] - 2s 10ms/step
10
157/157 [==============================] - 2s 9ms/step
10
157/157 [==============================] - 2s 9ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 2s 10ms/step
10
157/157 [==============================] - 2s 9ms/step
10
157/157 [==============================] - 2s 10ms/step
10
157/157 [==============================] - 2s 10ms/step
10
157/157 [==============================] - 1s 9ms/step
10
157/157 [==============================] - 2s 9ms/step
10
157/157 [==============================] - 2s 9ms/step
10
157/157 [==============================] - 2s 10m

  4%|▍         | 2/45 [3:06:30<62:44:22, 5252.61s/it]

...

 42%|████▏     | 19/45 [28:48:10<45:05:50, 6244.24s/it]

(38, 0.2, 125)
...
(16931, 38, 14) (16931, 1) (100, 38, 14)
4/4 [==============================] - 1s 6ms/step
(10, 38, 14) (10, 1)
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
1

 44%|████▍     | 20/45 [30:25:38<42:32:15, 6125.40s/it]

(38, 0.2, 130)
...
(16931, 38, 14) (16931, 1) (100, 38, 14)
4/4 [==============================] - 1s 10ms/step
(10, 38, 14) (10, 1)
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 7ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 8ms/step
10
157/157 [==============================] - 1s 7ms/step


: 

In [ ]:
df_metrics